<h1>Working with text!</h1>

<h3>Simple sentiment analysis</h3>
Compute the proportion of positive and negative words in a text


<h2>Corpus of positive and negative words</h2>

In [ ]:
p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'

In [ ]:
# let's see how the lists look
import requests
words = requests.get(p_url).content.decode('latin-1')
print(words)

In [ ]:
def get_words(url):
    import requests
    words = requests.get(url).content.decode('latin-1')

    word_list = words.split('\n')
    index = 0
    # Loop through the words/lines and remove what's not a word
    while index < len(word_list):
        word = word_list[index]
        if ';' in word or not word:
            word_list.pop(index)
        else:
            index+=1
    return word_list

#Get lists of positive and negative words
positive_words = get_words(p_url)
negative_words = get_words(n_url)
print(positive_words)


<h3>Interesting analysis of presidential candidates debates</h3>

In [ ]:
def remove_punctuation(word):
    if word and ((word[-1] >= 'a' and word[-1]<='z') or (word[-1] >= 'A' and word[-1]<='Z')):
        return word
    elif word:
        return word[:-1]
    else:
        return word

In [ ]:
word = "word."
print(word[-1] >= 'a')
#print(ord(word[-1]), ord('a'), ord('b'))
#(word[-1] >= 'a' and word[-1]<='z') or (word[-1] >= 'A' and word[-1]<='Z')

In [ ]:
#Get debate texts
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
    words = data.split()
    cpos=0
    cneg=0
    for word in words:
        word = remove_punctuation(word)
        if word in positive_words:
            cpos += 1
        if word in negative_words:
            cneg += 1
    total_words = len(words)
    ppct = cpos/total_words * 100
    npct = cneg/total_words * 100
    pos_neg_ratio = cpos/cneg
    print("%-10s\t%7d\t%2.2f\t%2.2f\t%2.2f"%(text,len(words),ppct,npct,pos_neg_ratio))


<h3>Let's see if the debate gets more positive or negative as it proceeds</h3>

In [ ]:
#First let's read all the texts
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
all_texts = dict()
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
        all_texts[text] = data
all_texts

<h2>nltk: Python's natural language toolkit</h2>

In [ ]:
#!pip install nltk --upgrade
#import nltk #The nltk library
#nltk.download() or nltk.download_shell()

<b>token:</b> A sequence (or group) of characters of interest. For e.g., in the below analysis, a token = a word
<li>Generally: A token is the base unit of analysis</li>
<li>So, the first step is to convert text into tokens and nltk text object</li>

<i>sent_tokenize: constructs sentences</i><br>

In [ ]:
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
text = all_texts[debate_texts[4]]
sents = sent_tokenize(text)
pos_neg_ratio_list = list()
cumpos=0
cumneg=0

for sent in sents:
    for word in sent.split():
        if word in positive_words:
            cumpos+=1
        if word in negative_words:
            cumneg+=1
    try:
        pos_neg_ratio_list.append(cumpos/cumneg)
    except:
        pos_neg_ratio_list.append(0)
plt.plot(pos_neg_ratio_list)

In [ ]:
#Let's functionalize this
def get_pos_neg_ratios(text):
    sents = sent_tokenize(text)
    pos_neg_ratio_list = list()
    cumpos=0
    cumneg=0
    for sent in sents:
        for word in sent.split():
            if word in positive_words:
                cumpos+=1
            if word in negative_words:
                cumneg+=1
        try:
            pos_neg_ratio_list.append(cumpos/cumneg)
        except:
            pos_neg_ratio_list.append(0)
    return pos_neg_ratio_list



<h3>Now we can do a side by side comparison for all three debates</h3>

In [ ]:
#First let's read all the texts
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
all_texts = dict()
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
        all_texts[text] = data


In [ ]:
#Set up side by side plots
COL_NUM = 2
ROW_NUM = 3
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))    
for i in range(len(debate_texts)):
    text = all_texts[debate_texts[i]]
    disp_list = get_pos_neg_ratios(text)
    ax = axes[i//2, i%2]
    ax.set_title(debate_texts[i])
    ax.plot(disp_list)
    ax.axis('off')
plt.show()

<h2>Creating a wordcloud.</h2>

In [ ]:
# !pip install wordcloud

In [ ]:
#First let's collect all the text into strings
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
all_texts = dict()
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
        all_texts[text] = data


<li>STOPWORDS: a list of common words that word cloud ignores
<li>If you need to ignore other words, you must explicitly delete them from your text

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1000).generate(all_texts['clinton.1'])

plt.imshow(wordcloud)
plt.axis('off')
plt.show()

<li>Let's remove the speaker markers from the texts
<li>Then we'll render our wordclouds in a 3x2 grid so that we can compare debate to debate

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Remove unwanted words
DELETE_WORDS = ['CLINTON','TRUMP','HOLT','WARREN','COOPER','RADDATZ','said']
def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ')
    return text_string

#Set up side by side clouds
COL_NUM = 2
ROW_NUM = 3
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
for i in range(0,len(debate_texts)):
    text_string = remove_words(all_texts[debate_texts[i]])
    ax = axes[i//2, i%2]
    ax.set_title(debate_texts[i])
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1000).generate(text_string)
    ax.imshow(wordcloud)
    ax.axis('off')
plt.show()


<li>Let's junk all small words
<li>We need to do some pre-processing of the text to see what words are junkable. Add those to DELETE_WORDS
<li>We'll also control the number of words that we're going to show

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Remove unwanted words
DELETE_WORDS = ['CLINTON','TRUMP','HOLT','WARREN','COOPER','RADDATZ']
def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ')
    return text_string

#Remove short words
def remove_short_words(text_string,min_length = 5):
    word_list = text_string.split()
    for word in word_list:
        if len(word) < min_length:
            text_string = text_string.replace(' '+word+' ',' ',1)
    return text_string


#Set up side by side clouds
COL_NUM = 2
ROW_NUM = 3
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
for i in range(0,len(debate_texts)):
    text_string = remove_words(all_texts[debate_texts[i]])
    text_string = remove_short_words(text_string)
    ax = axes[i//2, i%2]
    ax.set_title(debate_texts[i])
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1000,max_words=20).generate(text_string)
    ax.imshow(wordcloud)
    ax.axis('off')
plt.show()


In [ ]:
import nltk 
#so that graphs render inside the browser
%matplotlib inline 
import numpy as np #because numpy underlies everything and we never know when we'll need it!

In [ ]:
random_text = """
polonius to Laertes: Neither a borrower nor a lender be, 
For loan oft loses both itself and friend, 
And borrowing dulls the edge of husbandry.
This above all: to thine own self be true,
And it must follow, as the night the day,
Thou canst not then be false to any man.
Farewell. My blessing season this in thee.
"""


<i>word_tokenize: constructs word tokens</i>

In [ ]:
from nltk import sent_tokenize,word_tokenize
sentences = nltk.Text(sent_tokenize(random_text))
print(len(sentences))
words = nltk.Text(word_tokenize(random_text))
print(len(words))

<h3>nltk contains a large number of pre-tokenized data</h3>

In [ ]:
from nltk.book import *

In [ ]:
#text4[0:100]
len(text4)

In [ ]:
type(text4)

<h2>We can import our own texts into (our!) nltk corpus using the PlainTextCorpusReader</h2>

In [ ]:
import nltk
from nltk.corpus import PlaintextCorpusReader
corpus_root='Class 9 - data'
clinton_filename='clinton.*'
trump_filename='trump.*'
clinton_data = PlaintextCorpusReader(corpus_root,clinton_filename,encoding='latin-1')
trump_data = PlaintextCorpusReader(corpus_root,trump_filename,encoding='latin-1')
clinton_content = nltk.Text(clinton_data.words())
trump_content = nltk.Text(trump_data.words())

<h3>Basic analysis</h3>

<b>Complexity</b>. Judging a text by how hard it is to read!
We can look at the average word length, the average sentence length, and the number of different
words used (vocabulary) as a percentage of the length of the text



<h4>Complexity factors:</h4>
<li>average word length: longer words adds to complexity
<li>average sentence length: longer sentences are more complex (unless the text is rambling!)
<li>vocabulary: the ratio of unique words used to the total number of words (more variety, more complexity)

In [ ]:
string_version=' '.join(text4)
num_chars=len(string_version)
num_words=len(word_tokenize(string_version))
num_sentences=len(sent_tokenize(string_version))
vocab = {x.lower() for x in word_tokenize(string_version)}
print(num_chars,int(num_chars/num_words),int(num_words/num_sentences),len(vocab)/num_words)

In [ ]:
#Let's functionalize this for reuse
def get_complexity(text):
    num_chars=len(text)
    num_words=len(word_tokenize(text))
    num_sentences=len(sent_tokenize(text))
    vocab = {x.lower() for x in word_tokenize(text)}
    return len(vocab),int(num_chars/num_words),int(num_words/num_sentences),len(vocab)/num_words

In [ ]:
#And test it
print(get_complexity(string_version))

In [ ]:
#Wonder how that compares with Polonious' speech above?
print(get_complexity(random_text))

In [ ]:
# Hmm. Polonius speech uses more vocabulary than do our Presidents 
# (but uses shorter words and shorter sentences)!
# What about Jane Austen and monty python?
string_version_austen=' '.join(text2)
print(get_complexity(string_version_austen))
string_version_monty=' '.join(text6)
print(get_complexity(string_version_monty))

<h3>Let's compare austen, montypython, past presidents, and our two current candidates for president</h3>

In [ ]:
texts = (('Presidents',text4),('Austen',text2),('MontyPython',text6),('Clinton',clinton_content),('Trump',trump_content)) 
print('%14s\t%14s\t%14s\t%14s\t%14s'%('Text','Vocabulary','Word Length','Sentence Length','Word Variety'))
for text in texts:
    vocab,word_len,sen_len,variety = get_complexity(' '.join(text[1]))
    print('%14s\t%14d\t%14d\t%14d\t%7.2f'%(text[0],vocab,word_len,sen_len,variety))
    

In [ ]:
#Wonder if the complexity of Presidential inaugural addresses have changed over time

early_text = string_version[:392533] #approximately the first half
later_text = string_version[392533:] #approximately the second half
print(get_complexity(early_text))
print(get_complexity(later_text))

<h4>Sentences are getting shorter but word size and vocab ratio seem unchanged</h4>

<h4>We can drill down and look at sentence length for each individual inaugural address in temporal order to see how sentence length has changed from Washington to Obama (2009)</h4>

In [ ]:
#Import individual files from the corpus
from nltk.corpus import inaugural
sentence_lengths = list()
print(inaugural.fileids())
for fileid in inaugural.fileids():
    sentence_lengths.append(get_complexity(' '.join(inaugural.words(fileid)))[2])
plt.plot(sentence_lengths)

<h4>Let's see if 'debate practice' helped our candidates improve the complexity of their sentences</h4>

In [ ]:
for fileid in clinton_data.fileids():
    print(fileid,get_complexity(' '.join(clinton_data.words(fileid))))
    
for fileid in trump_data.fileids():
    print(fileid,get_complexity(' '.join(trump_data.words(fileid))))

<h3>Practice problem 1 ...</h3>

<h3>Practice problem 2...</h3>

<h2>Part of Speech tagging</h2>
<h4>POS tagging is useful because we get some sense of how a word is used</h4>
<h4>For example: We might want to see if the candidates differ on how they describe stuff. More positive or more negative</h4>
<h4>And that may be different from the  positive/negative ratios of the overall text</h4>

In [ ]:
nltk.pos_tag('I had a great spring break. I travelled with my family to Costa Rica and avoided the snow storm in New York'.split())

In [ ]:
#Get a list of all tags for reference
import nltk
nltk.help.upenn_tagset()

In [ ]:
words = nltk.pos_tag(text4)
words

<h4>Let's see how they differ on the use of adjectives (JJ, JJR, JJS) </h4>

In [ ]:
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
    words = nltk.pos_tag(data.split())
    cpos=0
    cneg=0
    for word in words:
        if not word[1] in ['JJ','JJR','JJS']:
            continue
        if word[0] in positive_words:
            cpos += 1
        if word[0] in negative_words:
            cneg += 1
    total_words = len(words)
    ppct = cpos/total_words * 100
    npct = cneg/total_words * 100
    pos_neg_ratio = cpos/cneg
    print("%10s\t%7d\t%2.2f\t%2.2f\t%2.2f"%(text,len(words),ppct,npct,pos_neg_ratio))


In [ ]:
#For comparison
debate_texts = ['clinton.1','trump.1','clinton.2','trump.2','clinton.3','trump.3']
for text in debate_texts:
    with open('Class 9 - data/'+text,'r') as f:
        data = f.read()
    words = data.split()
    cpos=0
    cneg=0
    for word in words:
        word = remove_punctuation(word)
        if word in positive_words:
            cpos += 1
        if word in negative_words:
            cneg += 1
    total_words = len(words)
    ppct = cpos/total_words *100
    npct = cneg/total_words * 100
    pos_neg_ratio = cpos/cneg
    print("%10s\t%7d\t%2.2f\t%2.2f\t%2.2f"%(text,len(words),ppct,npct,pos_neg_ratio))


<h3>Finding entity names in a text</h3>
<li>ne_chunk_sents takes a list of POS tagged sentences and returns a named entity tree with groups of NNP's and sentence fragments as the leaves
<li>NNP groups are extracted by seeing if they occur in the same sentence

In [ ]:
def get_entity_names(text):
    sentences = nltk.sent_tokenize(text)
    print(sentences)
    #tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    #print(tokenized_sentences)
    #tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    #print(tagged_sentences)
    #chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    #for c in chunked_sentences:
    #    print(c)

In [ ]:
#Clinton in debate 1
get_entity_names(all_texts['clinton.1'])

In [ ]:
def get_entity_names(text):
    sentences = nltk.sent_tokenize(text)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    
    def extract_entity_names(t):
        entity_names = []

        if hasattr(t, 'label') and t.label:
            if t.label() == 'NE':
                entity_names.append(' '.join([child[0] for child in t]))
            else:
                for child in t:
                    entity_names.extend(extract_entity_names(child))

        return entity_names

    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extract_entity_names(tree))

    return set(entity_names)

In [ ]:
#Clinton in debate 1
get_entity_names(all_texts['clinton.1'])

<h4>Let's see what the differences between the two candidates is on named entities</h4>


In [ ]:
clinton_texts = all_texts['clinton.1'] + all_texts['clinton.2'] + all_texts['clinton.3']
clinton_ne = get_entity_names(clinton_texts)
trump_texts = all_texts['trump.1'] + all_texts['trump.2'] + all_texts['trump.3']
trump_ne = get_entity_names(trump_texts)

In [ ]:
clinton_ne - trump_ne

In [ ]:
trump_ne - clinton_ne

<h2>Frequency Distributions</h2>

In [ ]:
fdist_clinton = FreqDist(clinton_content)
fdist_trump = FreqDist(trump_content)
fdist_clinton

In [ ]:
#Let's look at the top 50 words
fdist_clinton.plot(50)

<h4>Not very helpful! Let's see what proportion of Clinton words are in the top 50</h4>

In [ ]:
print(len(clinton_content))
fdist_clinton.plot(50,cumulative=True)

In [ ]:
#Let's get rid of common words
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
print(stop)
#fdist_clinton = FreqDist(clinton_content)
#for word in stop:
#    if fdist_clinton.get(word):
#        del fdist_clinton[word]
#        
#len(fdist_clinton)      

In [ ]:
#Let's functionalize this
def fdist_stop_words(corpus,delete_words):
    from nltk.corpus import stopwords
    import nltk
    stop = set(stopwords.words('english'))
    for word in delete_words:
        stop.add(word)
    fdist = FreqDist(corpus)
    for word in stop:
        if fdist.get(word):
            del fdist[word]
    return fdist

In [ ]:
fdist_stop_words(clinton_content,DELETE_WORDS).plot(50)

In [ ]:
DELETE_WORDS

In [ ]:
#Not helpful again. Let's look only at long words
clinton_long = [x.lower() for x in clinton_content if len(x)>8 and x.isalpha()]
trump_long = [x.lower() for x in trump_content if len(x)>8 and x.isalpha()]
DELETE_WORDS = ['something','everything']
    
fdist_clinton = fdist_stop_words(clinton_long,DELETE_WORDS)
fdist_trump = fdist_stop_words(trump_long,DELETE_WORDS)
fdist_trump.most_common(10)
#fdist_trump.plot(10)

In [ ]:
fdist_clinton.most_common(10)
fdist_clinton.plot(10)

<h4>We can use these distributions to look at differences between the candidates</h4>

In [ ]:
fdist_clinton

In [ ]:
clinton_words = {w for w in fdist_clinton if fdist_clinton[w]>10}
trump_words = {w for w in fdist_trump if fdist_trump[w]>10}
print(clinton_words - trump_words)
trump_words - clinton_words

Words can mean different things in different contexts so let's try a bit of context analysis<p>
<b>Concordance</b>: Occurrences of a word with some context (text around the word)

In [ ]:
trump_content.concordance('obamacare')

In [ ]:
clinton_content.concordance('obamacare')

<h4>We can look at who uses a word in a positive context and who in a negative context</h4>
But we have to write our own concordance function for this

In [ ]:
def concordance(text,word,width):
    locs = [loc for loc, x in enumerate(text) if x.lower() == word.lower()]
    fragments = list()
    for loc in locs:
        start = max(0,loc-width)
        end = min(len(text),loc+width)
        fragments.append(' '.join(text[start:end]))
    return fragments
concordance(trump_content,'immigrants',10)

<h4>Now we can write a function to get sentiment for a word</h4>

    

In [ ]:
def get_concordance_sentiment(text,word,width):
    sentences = concordance(text,word,width)
    pos=0
    neg=0
    for sent in sentences:
        for word in sent.split():
            if word in positive_words:
                pos+=1
            if word in negative_words:
                neg+=1
    if not pos and not neg:
        return 1
    if pos and not neg:
        return 10
    else:
        return pos/neg


In [ ]:
print(get_concordance_sentiment(clinton_content,'immigrants',10))
print(get_concordance_sentiment(trump_content,'immigrants',10))

In [ ]:
print(get_concordance_sentiment(clinton_content,'obamacare',10))
print(get_concordance_sentiment(trump_content,'obamacare',10))

In [ ]:
print(get_concordance_sentiment(clinton_content,'people',10))
print(get_concordance_sentiment(trump_content,'people',10))

In [ ]:
print(get_concordance_sentiment(clinton_content,'obama',10))
print(get_concordance_sentiment(trump_content,'obama',10))

In [ ]:
print(get_concordance_sentiment(clinton_content,'donald',10))
print(get_concordance_sentiment(trump_content,'hillary',10))

In [ ]:
print(get_concordance_sentiment(clinton_content,'i',10))
print(get_concordance_sentiment(trump_content,'i',10))

<li>Similar. What words appear in the same context as the given word
<li>When words are used in the same context, they are likely to give us the sense in which the word is used

In [ ]:
clinton_content.similar('people')

In [ ]:
trump_content.similar('people')

In [ ]:
text4.concordance('people')

In [ ]:
get_concordance_sentiment(text4,'people',10)

In [ ]:
clinton_content.similar('americans')

In [ ]:
trump_content.similar('americans')

<h4>Earlier we saw that Clinton and Trump stressed different words</h4>

In [ ]:
print(get_concordance_sentiment(clinton_content,'government',10))  # She said
print(get_concordance_sentiment(trump_content,'politicians',10)) # He said

<h4>Dispersion plots show how word use changes over the course of the text</h4>

In [ ]:
trump_content.dispersion_plot(['she','Secretary','Hillary'])

In [ ]:
clinton_content.dispersion_plot(['he', 'Trump','Donald'])

<h4>We can see how the thinking of our presidents has changed</h4>

In [ ]:
text4.dispersion_plot(["citizens", "democracy", "freedom", "duties", "Americans",'independence','God'])

In [ ]:
#Question: The main characters in Sense and Sensibility are Elinor, Marianne, Edward, and Willoughby. 
#Who are more important to the story, the men or the women?
text2.dispersion_plot(['Elinor','Marianne','Edward','Willoughby'])

In [ ]:
#How about a comparison between early versus late uses of citizen?
early_text_tokens.similar('citizen')
print('----')
later_text_tokens.similar('citizen')

<li>Hmm. states seems to be replaced by nation or country 
<li>Law and government are out. 




<h4>Common contexts. The set of contexts that are common for a list of words</h4>

In [ ]:
early_text_tokens.common_contexts(['citizen','government'])
print('----')
later_text_tokens.common_contexts(['citizen','government'])

<h4>Let's look at the inaugural speeches again:</h4>
<li>Since the inaugural speeches are temporally arranged, we can see the temporal occurrence and distribution of words over time

In [ ]:
early_text_tokens.similar('duties')

In [ ]:
later_text_tokens.similar('duties')

In [ ]:
text4.concordance('duties')

<li>Generally, we're interested in: the frequency of a word and in the most frequent words
</li>Let's look at frequency distributions

In [ ]:
fdist1 = FreqDist(text4)
fdist1.most_common(10) #Try 50, 100

In [ ]:
#Not very helpful. Let's see what proportion of the words in the top 50 make up the text
fdist1.plot(50,cumulative=True)

In [ ]:
len(text4) #Approximately half the words in the text are in the top 50. Most are useless!

<h2>Bi-grams and n-grams</h2>

<li>Individual words don't tell us a whole lot. So we work with pairs, triads, etc. of words that go together.
<li>bigram: a pair of words that go togther. 'American people', 'God bless','Soviet Union'
<li>Collocations: Frequent bigrams in a text

In [ ]:
clinton_content.collocations()

In [ ]:
trump_content.collocations()

<li>Collocations are computed by a submodule of nltk. 
<li>All pairs of words are examined and a strength (using the relative frequency of co-occurence) is assigned to each word.
<li>PMI: Pointwise Mutual Information (uses conditional probabilities based on the relative frequency to compute a measure of association)
<li>We can use this module to identify all bigrams (and trigrams)

In [ ]:
from nltk.collocations import *

In [ ]:
finder = nltk.BigramCollocationFinder.from_words(clinton_content)
bigram_measures = nltk.collocations.BigramAssocMeasures()
ignored_words = nltk.corpus.stopwords.words('english') + ['CLINTON','...'] 
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.apply_freq_filter(5)
print(finder.nbest(bigram_measures.pmi,50))


In [ ]:
finder = nltk.BigramCollocationFinder.from_words(trump_content)
bigram_measures = nltk.collocations.BigramAssocMeasures()
ignored_words = nltk.corpus.stopwords.words('english') + ['LAUGHTER',')',',','CROSSTALK'] #Set up a list of words to ignore
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.apply_freq_filter(5)
print(finder.nbest(bigram_measures.pmi,50))

<h4>By adjusting the window size, nltk can find associations even if there are intervening words</h4>

In [ ]:
finder = nltk.BigramCollocationFinder.from_words(trump_content,window_size=10)
bigram_measures = nltk.collocations.BigramAssocMeasures()
ignored_words = nltk.corpus.stopwords.words('english') + ['LAUGHTER',')',',','CROSSTALK'] #Set up a list of words to ignore
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.apply_freq_filter(5)
print(finder.nbest(bigram_measures.pmi,50))

In [ ]:
finder = nltk.BigramCollocationFinder.from_words(clinton_content,window_size=10)
bigram_measures = nltk.collocations.BigramAssocMeasures()
ignored_words = nltk.corpus.stopwords.words('english') + ['LAUGHTER',')',',','CROSSTALK'] #Set up a list of words to ignore
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.apply_freq_filter(5)
print(finder.nbest(bigram_measures.pmi,50))

<h4>Trigrams: Groups of 3</h4>

In [ ]:
finder = nltk.TrigramCollocationFinder.from_words(clinton_content)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
scored = finder.score_ngrams(trigram_measures.raw_freq)
sorted(finder.nbest(trigram_measures.pmi,15))

In [ ]:
finder = nltk.TrigramCollocationFinder.from_words(trump_content)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
scored = finder.score_ngrams(trigram_measures.raw_freq)
sorted(finder.nbest(trigram_measures.pmi,15))

<h3>Using bigrams and trigrams</h3>
<ol>
<li>Sentiment analysis on concepts
<li>Construct networks of words to identify broader concepts (tune in next week ...)

<h2>Conditional Frequency Distributions</h2>

<li>A conditional frequency distribution allows us to compare texts in different categories by counting the occurrence of words in each category (or text)
<li>For example, the inaugural addresses are divided into multiple files for each address arranged temporally
<li>We can use this time element to count words and see how usage changes across time

In [ ]:
clinton_data.fileids() 

In [ ]:
cfd_clinton=nltk.ConditionalFreqDist((target,fileid) 
                             for fileid in clinton_data.fileids() 
                             for w in clinton_data.words(fileid)
                             for target in ['country','border']
                             if w.lower().startswith(target))

In [ ]:
cfd_clinton.plot()

In [ ]:
cfd_trump=nltk.ConditionalFreqDist((target,fileid) 
                             for fileid in trump_data.fileids() 
                             for w in trump_data.words(fileid)
                             for target in ['country','border']
                             if w.lower().startswith(target))

In [ ]:
cfd_trump.plot()

In [ ]:
cfd_trump.tabulate()

In [ ]:
cfd_clinton.tabulate()

<h4>Let's see how inward/outward looking the nation has gotten since Washington's inauguration

In [ ]:
inaugural.fileids() 

In [ ]:
cfd_inaugural=nltk.ConditionalFreqDist((target,fileid) 
                             for fileid in inaugural.fileids() 
                             for w in inaugural.words(fileid)
                             for target in ['world','nation']
                             if w.lower().startswith(target))
cfd_inaugural.plot()

<h2>NLTK resources:</h2>
<h3>ntlk documentation:</h3> http://www.nltk.org/api/nltk.html
<h3>Commands cheat sheet</h3> https://blogs.princeton.edu/etc/files/2014/03/Text-Analysis-with-NLTK-Cheatsheet.pdf
<h3>nltk book</h3>http://www.nltk.org/book/